# Introduction aux Physics-informed neural networks (PINNs)

Fortement inspiré des travaux de : Ben Moseley, 2022
Aurelien Plyer 2022

Pour plus d'information sur les PINN's la lecture des papiers suivant est plus que souhaité [here](https://ieeexplore.ieee.org/document/712178) et [here](https://www.sciencedirect.com/science/article/pii/S0021999118307125).


## Objectif du Notebook


Ce notebook à pour objectif de présenter les PINN's au travers
d'un cas applicatif simple d'une masse aoscilente.
L'implémentation est fait en Pytorch.

Les cas d'études permettrons de mettre en pratique les différents régimes de fonctionnement d'un PINN's

In [ ]:
!pip install matplotlib-inline -qq

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(123)


In [ ]:
from IPython import display
from matplotlib import pyplot as plt

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')
plt.style.use('dark_background')


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True

In [ ]:
device

## Aperçu du problème

Nous allons utiliser un PINN pour résoudre des problèmes liés à l'**oscillateur harmonique amorti**.

Nous souhaitons modéliser le déplacement de la masse sur un ressort  en fonction du temps.

Il s'agit d'un problème de physique canonique, où le déplacement, $u(t)$, de l'oscillateur en fonction du temps peut être décrit par l'équation différentielle suivante :

$$
m \dfrac{d^2 u}{d t^2} + \mu \dfrac{d u}{d t} + ku = 0~,
$$

où $m$ est la masse de l'oscillateur, $\mu$ est le coefficient de frottement et $k$ est la constante du ressort.

Nous nous concentrerons sur la résolution du problème dans l'état **sous-amorti**, c'est-à-dire lorsque l'oscillation est lentement amortie par le frottement (comme le montre l'animation ci-dessus).

Mathématiquement, cela se produit lorsque :
$$
\delta < \omega_0~,~~~~~\mathrm{where}~~\delta = \dfrac{\mu}{2m}~,~\omega_0 = \sqrt{\dfrac{k}{m}}~.
$$

En outre, nous considérons les conditions initiales suivantes du système :

$$
u(t=0) = 1~~,~~\dfrac{d u}{d t}(t=0) = 0~.
$$

Pour ce cas particulier, la solution exacte est connue et donnée par :

$$
u(t) = e^{-\delta t}(2 A \cos(\phi + \omega t))~,~~~~~\mathrm{with}~~\omega=\sqrt{\omega_0^2 - \delta^2}~.
$$


## Plan du notebook


Il y a **plusieurs tâches** liées à l'oscillateur harmonique pour lesquelles nous utiliserons un PINN :

1. Nous allons **régresser** un réseau de neuronnes pour estimer une fonction continue et différentiable à partir des données
1.  Nous **simulerons** le système à l'aide d'un réseau neuronal, compte tenu de ses conditions initiales.
1. Nous **inverserons** les paramètres sous-jacents du système à l'aide d'un réseau neuronal, compte tenu de certaines observations bruitées du déplacement de l'oscillateur.
1. Nous étudierons dans quelle mesure le PINN **s'adapte** à des oscillations de fréquence plus élevée et ce qui peut être fait pour améliorer sa convergence.

## Code global

On commence par coder la solution exacte au problème de la masse avec ressort :

In [ ]:
def exact_solution(d, w0, t):
    "Defines the analytical solution to the under-damped harmonic oscillator problem above."
    assert d < w0
    w = np.sqrt(w0**2-d**2)
    phi = np.arctan(-d/w)
    A = 1/(2*np.cos(phi))
    cos = torch.cos(phi+w*t)
    exp = torch.exp(-d*t)
    u = exp*2*A*cos
    return u


On définit ici un simple réseau MLP à x couches caché et K réseau par couches :

In [ ]:

class FCN(nn.Module):
    "Defines a standard fully-connected network in PyTorch"
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)

    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

# Utilisation d'un réseau de neuronne comme un régresseur sur les données

Dans ce premier test on utilisera le réseau de neuronne simplement pour représenter le jeux de donnée. On est alors dans un cas de simple régression.

La littérature appel aussi cela une représentation neural implicite, ce qui donnera en vision par ordinnateur 3D la naissance des Nerf par exemple (Neural Radiance Fields).

Le problème se traduit donc comme suit :

- Inputs: Des points d'apprentissages $\{(u_i, t_i)\}_{i=1..N}$
- Outputs: un réseau de neuronne qui approxime la fonction $u(t)$

Cette approximation possède deux avantage sur les points d'apprentissage :
- elle est continue
- elle est dérivable automatiquement de manière exacte



## Boucle d'apprentissage sur les points sans bruit

In [ ]:
# Number of trainning points
N_train = 30
N_total = 300
factor_train = 0.5

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)
# train the PINN
d, w0 = 2, 20
mu, k = 2*d, w0**2

t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

# we cheat here to take only the first half of data
idx = np.random.choice(int(factor_train*N_total), N_train, replace=False)

t_train = t_test[idx]
u_train = u_exact[idx]

optimiser = torch.optim.Adam(pinn.parameters(),lr=1e-3)
fig = None

for i in range(15001):
    optimiser.zero_grad()
    u_pred = pinn(t_train.to(device))
    loss = torch.mean((u_pred-u_train.to(device))**2)# use mean squared error
    loss.backward()
    optimiser.step()
    if i % 500 == 0:
        u = pinn(t_test.to(device)).detach().cpu()
        if fig is None:
            fig = plt.figure(figsize=(8,3))
        fig.clf()
        plt.scatter(t_train,
                    u_train, s=20, lw=0, color="tab:green", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="NN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        display.display(fig)
        display.clear_output(wait=True)

On observe ici que même si le NN à extrêmement bien estimé la fonction sur la zone de nos observation, dès lors qu'on s'en éloigne l'estimation ne ressemble à rien de physique.

C'est un point important à avoir en tête lors de l'apprentissage de réseaux de neuronnes. En général on est trés bon pour faire de l'interpolation de la variété des données d'apprentissage, par contre dès lors qu'on s'écarte de cette variété de départ l'extrapolation se passe trés mal.

Cela à donnée lieu à de nombreux domaine d'étude comme le *transfert learning* et le *continuous learning* qui vont chercher à repousser ces limites.

## cas de données bruités

Bien évidement apprendre sur un jeux de données sans bruit c'est tricher. Regardons ce qu'il se passe si on ajoute un peut de bruit sur les données. Hésitez pas a jouer avec le niveau de bruit (noise_level)

In [ ]:
# Number of trainning points
N_train = 30
N_total = 300
noise_level = 0.04
factor_train = 0.8
d, w0 = 2, 20

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)
mu, k = 2*d, w0**2

t_test = torch.linspace(0,1,N_total).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

# we cheat here to take only the first half of data
idx = np.random.choice(int(factor_train*N_total), N_train, replace=False)

t_train = t_test[idx]
u_train = u_exact[idx]+ noise_level*torch.randn_like(u_exact[idx])

optimiser = torch.optim.Adam(pinn.parameters(),lr=1e-3)
fig = None

for i in range(15001):
    optimiser.zero_grad()
    u_pred = pinn(t_train.to(device))
    loss = torch.mean((u_pred-u_train.to(device))**2)# use mean squared error
    loss.backward()
    optimiser.step()
    if i % 500 == 0:
        u = pinn(t_test.to(device)).detach().cpu()
        if fig is None:
            fig = plt.figure(figsize=(8,3))
        fig.clf()
        plt.scatter(t_train,
                    u_train, s=20, lw=0, color="tab:green", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="NN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        display.display(fig)
        display.clear_output(wait=True)

On remarque qu'au fure et a mesure des itérations le model overfit les données.

**Question :** explorer les hyper-paramètres afin de voir si vous ne pouvez pas limiter l'overfitting.

# Simulation de phénomène avec un PINN's


#### Tâche

La première tâche consiste à utiliser un PINN pour **simuler** le système.

Plus précisément, nos entrées et sorties sont les suivantes :

- Entrées : équation différentielle sous-jacente et conditions initiales du système
- Sorties : estimation de la solution, $u(t)$.

#### Approche

Le PINN est entraîné à approximer directement la solution de l'équation différentielle, c'est-à-dire

$$
u_{\mathrm{PINN}}(t;\theta) \approx u(t)~,
$$

où $\theta$ sont les paramètres libres du PINN.


#### Fonction de perte

Pour simuler le système, le PINN est entraîné avec la fonction de perte suivante :

$$
\mathcal{L}(\theta)=  \mathcal{L}_0(\theta) + \lambda_1 \mathcal{L}_1(\theta) + \lambda_2\mathcal{L}_2(\theta)
$$

où :  

$$
\mathcal{L}_0(\theta)= (u_{\mathrm{PINN}}(t=0;\theta)
$$

$$
\mathcal{L}_1(\theta) = \left(\frac{d\,u_{\mathrm{PINN}}}{dt}(t=0;\theta) - 0\right)^2
$$

$$
\mathcal{L}_2(\theta) =  \frac{1}{N} \sum^{N}_{i} \left( \left[ m\frac{d^2}{dt^2} + \mu \frac{d}{dt} + k \right] u_{\mathrm{PINN}}(t_{i};\theta)  \right)^2
$$


Pour cette tâche, nous utilisons $\delta=2$, $\omega_0=20$, et nous essayons d'apprendre la solution sur le domaine $\in [0,1]$.



#### Notes

Les deux premiers termes de la fonction de perte $\mathcal{L}_i(\theta)$, $i = \{0,1\}$  représentent la **condition de bords**, et servent à garantir que la solution apprise par le PINN correspond aux conditions initiales du système, à savoir $u(t=0)=1$ et $u'(t=0)=0$.

Le terme $\mathcal{L}_2$ est appelé **perte physique** et vise à garantir que la solution PINN obéit à l'équation différentielle sous-jacente à un ensemble de points d'apprentissage $\{t_i\}$ *(collocation points)* échantillonnés sur l'ensemble du domaine.

Les hyperparamètres, $\lambda_1$ et $\lambda_2$, sont utilisés pour équilibrer les termes de la fonction de perte, afin de garantir la stabilité pendant l'apprentissage.

L'autodifférenciation (`torch.autograd`) est utilisée pour calculer les gradients de la PINN par rapport à son entrée requise pour évaluer la fonction de perte. C'est très puissant !

Pour plus de détails sur `torch.autograd`, consultez le tutoriel [this](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#a-gentle-introduction-to-torch-autograd).

### Trainning loop

In [ ]:
number_of_physics_point = 30
lambda1, lambda2 = 1e-1, 1e-4
n_iter = 20001
d, w0 = 2, 20
N_tot = 300

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)
# define boundary points, for the boundary loss
t_boundary = torch.tensor(0.).view(-1,1).requires_grad_(True)
# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,number_of_physics_point).view(-1,1).requires_grad_(True)

# train the PINN
mu, k = 2*d, w0**2
t_test = torch.linspace(0,1,N_tot).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

optimiser = torch.optim.Adam(pinn.parameters(),lr=1e-3)
fig = None

for i in range(n_iter):
    optimiser.zero_grad()
    t_b = t_boundary.to(device)
    # compute boundary loss
    u = pinn(t_b)
    loss1 = (torch.squeeze(u) - 1)**2
    dudt = torch.autograd.grad(u, t_b, torch.ones_like(u).to(device), create_graph=True)[0]
    loss2 = (torch.squeeze(dudt) - 0)**2

    # compute physics loss
    t_p = t_physics.to(device)
    u = pinn(t_p)
    dudt = torch.autograd.grad(u, t_p, torch.ones_like(u).to(device), create_graph=True)[0]
    d2udt2 = torch.autograd.grad(dudt, t_p, torch.ones_like(dudt).to(device), create_graph=True)[0]
    loss3 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2 + lambda2*loss3
    loss.backward()
    optimiser.step()

    # plot the result as training progresses
    if i % 500 == 0:
        #print(u.abs().mean().item(), dudt.abs().mean().item(), d2udt2.abs().mean().item())
        u = pinn(t_test.to(device)).detach().cpu()
        if fig is None:
            fig = plt.figure(figsize=(8,3))
        fig.clf()
        plt.scatter(t_physics.detach()[:,0],
                    torch.zeros_like(t_physics)[:,0], s=20, lw=0, color="tab:green", alpha=0.6)
        plt.scatter(t_boundary.detach()[:,0],
                    torch.zeros_like(t_boundary)[:,0], s=20, lw=0, color="tab:red", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        display.display(fig)
        display.clear_output(wait=True)

## Regression physique avec du bruit

Cette fois-ci combinons la regression de donnée avec une fonction de perte physique pour apprendre une représentation neural contraint par la physique.


In [ ]:
lambda1 = 1e4
n_iter = 20001
d, w0 = 2, 20
lambda1 = 1e4
n_iter = 20001
d, w0 = 2, 20

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)

# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)

# train the PINN
_, k = 2*d, w0**2


# add mu to the optimiser
optimiser = torch.optim.Adam(list(pinn.parameters()),lr=1e-3)
for i in range(n_iter):
    optimiser.zero_grad()

    # compute physics loss
    t_p = t_physics.to(device)
    u = pinn(t_p)
    dudt = torch.autograd.grad(u, t_p, torch.ones_like(u).to(device), create_graph=True)[0]
    d2udt2 = torch.autograd.grad(dudt, t_p, torch.ones_like(dudt).to(device), create_graph=True)[0]
    loss1 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

    # compute data loss

    u = pinn(t_obs.to(device))
    loss2 = torch.mean((u - u_obs.to(device))**2)

    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2
    loss.backward()
    optimiser.step()


    # plot the result as training progresses
    if i % 500 == 0:
        if fig is None:
            fig = plt.figure(figsize=(11,3))
        fig.clf()
        u = pinn(t_test.to(device)).detach().cpu()
        plt.scatter(t_obs[:,0], u_obs[:,0], label="Noisy observations", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        plt.show()
        display.display(fig)
        display.clear_output(wait=True)


## inversion de paramètres par PINN's

#### Tâche

La deuxième tâche consiste à utiliser un PINN pour **inverser** les paramètres sous-jacents.

Plus précisément, nos entrées et sorties sont les suivantes

- Entrées : observations bruitées du déplacement de l'oscillateur
- Sorties : estimation de $\mu$, le coefficient de friction

#### Approche

Comme ci-dessus, le PINN est entraîné à approximer directement la solution de l'équation différentielle, c'est-à-dire

$$
u_{\mathrm{PINN}}(t;\theta) \approx u(t)~,
$$

où $\theta$ sont les paramètres libres de la PINN.

L'idée clé ici est de traiter $\mu$ comme un **paramètre apprenable** lors de l'apprentissage de la PINN - de sorte que nous simulons la solution et inversons pour ce paramètre.

#### Loss function

Le PINN est formé avec une fonction de perte légèrement différente

$$
\mathcal{L}(\theta, \mu)= \frac{1}{N} \sum^{N}_{i} \left( \left[ m\frac{d^2}{dt^2} + \mu \frac{d}{dt} + k \right] u_{\mathrm{PINN}}(t_{i};\theta)  \right)^2 + \frac{\lambda}{M} \sum^{M}_{j} \left( u_{\mathrm{PINN}}(t_{j};\theta) - u_{\mathrm{obs}}(t_{j}) \right)^2
$$






#### Notes

La fonction de perte comporte deux termes. Le premier est la **perte de physique**, formée de la même manière que ci-dessus, qui garantit que la solution apprise par la PINN est cohérente avec la physique connue.

Le second terme est appelé **perte de données**, et assure que la solution apprise par le PINN correspond aux observations (potentiellement bruitées) de la solution qui sont disponibles.

Nous avons supprimé les termes de perte aux limites, car nous ne les connaissons pas (nous ne disposons que des mesures observées du système).

Dans cette configuration, les paramètres PINN $\theta$ et $\mu$ sont **conjointement** appris pendant l'optimisation.

Là encore, l'autodifférenciation est notre amie et nous permettra de définir facilement ce problème !

In [ ]:
# first, create some noisy observational data

d, w0 = 2, 20
print(f"True value of mu: {2*d}")
t_obs = torch.rand(40).view(-1,1)*0.5
u_obs = exact_solution(d, w0, t_obs) + 0.04*torch.randn_like(t_obs)

plt.figure(figsize = (8,4))
plt.title("Noisy observational data")
plt.scatter(t_obs[:,0], u_obs[:,0])
t_test, u_exact = torch.linspace(0,1,300).view(-1,1), exact_solution(d, w0, t_test)
plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
_ = plt.show()

### Trainning loop

In [ ]:
lambda1 = 1e4
n_iter = 20001
d, w0 = 2, 20

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)

# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)

# train the PINN
_, k = 2*d, w0**2

# treat mu as a learnable parameter
mu = torch.nn.Parameter(1+torch.zeros(1, requires_grad=True, device = device))
mus = []

# add mu to the optimiser
optimiser = torch.optim.Adam(list(pinn.parameters())+[mu],lr=1e-3)
for i in range(n_iter):
    optimiser.zero_grad()

    # compute physics loss
    t_p = t_physics.to(device)
    u = pinn(t_p)
    dudt = torch.autograd.grad(u, t_p, torch.ones_like(u).to(device), create_graph=True)[0]
    d2udt2 = torch.autograd.grad(dudt, t_p, torch.ones_like(dudt).to(device), create_graph=True)[0]
    loss1 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

    # compute data loss

    u = pinn(t_obs.to(device))
    loss2 = torch.mean((u - u_obs.to(device))**2)

    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2
    loss.backward()
    optimiser.step()

    # record mu value
    mus.append(mu.item())

    # plot the result as training progresses
    if i % 500 == 0:
        if fig is None:
            fig = plt.figure(figsize=(11,3))
        fig.clf()
        u = pinn(t_test.to(device)).detach().cpu()
        plt.scatter(t_obs[:,0], u_obs[:,0], label="Noisy observations", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i} $\mu$ = %0.3f"%mu.item())
        plt.legend()
        plt.show()
        display.display(fig)
        display.clear_output(wait=True)

In [ ]:
plt.figure(figsize=(8,4))
plt.title("$\mu$")
plt.plot(mus, label="PINN estimate")
plt.hlines(2*d, 0, len(mus), label="True value", color="tab:green")
plt.legend()
plt.xlabel("Training step")
plt.show()

# Lien entre fréquence et échantillonage

Il y a un lien entre la fréquence du phénomène que le réseau de neuronne peut reconstruire et l'écartement des points de colocations, c'est ce que nous allons étudier ici.

#### Tâche

La tâche finale consiste à étudier la capacité du PINN à **s'adapter** à des oscillations de plus haute fréquence et ce qui peut être fait pour améliorer sa convergence.

Plus précisément, nous revenons à la simulation de la solution de l'oscillateur harmonique et augmentons sa fréquence, $\omega_0$.

#### Standard formulation



In [ ]:
number_of_physics_point = 1000
lambda1, lambda2 = 1e-1, 1e-4
n_iter = 20001
d, w0 = 2, 80
N_tot = 300

# define a neural network to train
pinn = FCN(1,1,32,3).to(device)
# define boundary points, for the boundary loss
t_boundary = torch.tensor(0.).view(-1,1).requires_grad_(True)
# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,number_of_physics_point).view(-1,1).requires_grad_(True)

# train the PINN
mu, k = 2*d, w0**2
t_test = torch.linspace(0,1,N_tot).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

optimiser = torch.optim.Adam(pinn.parameters(),lr=1e-3)
fig = None

for i in range(n_iter):
    optimiser.zero_grad()

    # compute boundary loss
    t_b= t_boundary.to(device)
    u = pinn(t_b)
    loss1 = (torch.squeeze(u) - 1)**2
    dudt = torch.autograd.grad(u, t_b, torch.ones_like(u).to(device), create_graph=True)[0]
    loss2 = (torch.squeeze(dudt) - 0)**2

    # compute physics loss
    t_p = t_physics.to(device)
    u = pinn(t_p)
    dudt = torch.autograd.grad(u, t_p, torch.ones_like(u).to(device), create_graph=True)[0]
    d2udt2 = torch.autograd.grad(dudt, t_p, torch.ones_like(dudt).to(device), create_graph=True)[0]
    loss3 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2 + lambda2*loss3
    loss.backward()
    optimiser.step()

    # plot the result as training progresses
    if i % 500 == 0:
        #print(u.abs().mean().item(), dudt.abs().mean().item(), d2udt2.abs().mean().item())
        u = pinn(t_test.to(device)).detach().cpu()
        if fig is None:
            fig = plt.figure(figsize=(8,3))
        fig.clf()
        plt.scatter(t_physics.detach()[:,0],
                    torch.zeros_like(t_physics)[:,0], s=20, lw=0, color="tab:green", alpha=0.6)
        plt.scatter(t_boundary.detach()[:,0],
                    torch.zeros_like(t_boundary)[:,0], s=20, lw=0, color="tab:red", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        display.display(fig)
        display.clear_output(wait=True)

Vous devriez constater que le PINN peine à converger, même si le nombre de points d'entraînement en physique est augmenté.

Il s'agit d'un problème plus difficile à résoudre pour le PINN, en partie à cause du **biais spectral** des réseaux neuronaux, ainsi que du fait qu'un plus grand nombre de points d'entraînement est nécessaire.

Une autre solution pourrait être d'utiliser des couches de transformation du type des [fourrier features](https://bmild.github.io/fourfeat/)

# Approche : formulation alternative de l'"ansatz

Pour accélérer la convergence, on peut **supposer quelque chose** à propos de la solution.

Par exemple, supposons que nous sachions d'après notre intuition physique que la solution est en fait sinusoïdale.

Alors, au lieu de faire en sorte que le PINN approxime directement la solution de l'équation différentielle, c'est-à-dire

$$
u_{\mathrm{PINN}}(t;\theta) \approx u(t)~,
$$
Nous utilisons plutôt le PINN dans le cadre d'un ansatz mathématique de la solution, c'est-à-dire
$$
\hat u(t; \theta, \alpha, \beta) = u_{\mathrm{PINN}}(t;\theta)  \sin (\alpha t + \beta) \approx u(t)~,
$$
où $\alpha, \beta$ sont traités comme des paramètres supplémentaires pouvant être appris.

En comparant cet ansatz à la solution exacte
$$
u(t) = e^{-\delta t}(2 A \cos(\phi + \omega t))
$$
Nous constatons que le PINN n'a plus qu'à apprendre la fonction exponentielle, ce qui devrait être un problème beaucoup plus facile.

Une fois encore, l'autodifférenciation nous permet de différencier facilement par le biais de cet ansatz pour former le PINN !

In [ ]:
n_iter = 20001
a_init = 70
lambda1, lambda2 = 1e-1, 1e-3
d, w0 = 2, 60# note w0 is higher!


# define a neural network to train
pinn = FCN(1,1,32,3)

# define additional a,b learnable parameters in the ansatz
a = torch.nn.Parameter(a_init*torch.ones(1, requires_grad=True))
b = torch.nn.Parameter(torch.ones(1, requires_grad=True))

# define boundary points, for the boundary loss
t_boundary = torch.tensor(0.).view(-1,1).requires_grad_(True)

# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,60).view(-1,1).requires_grad_(True)

# train the PINN
mu, k = 2*d, w0**2
t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(d, w0, t_test)
optimiser = torch.optim.Adam(list(pinn.parameters())+[a,b],lr=1e-3)
fig = None

for i in range(n_iter):
    optimiser.zero_grad()

    # compute boundary loss
    u = pinn(t_boundary)*torch.sin(a*t_boundary+b)
    loss1 = (torch.squeeze(u) - 1)**2
    dudt = torch.autograd.grad(u, t_boundary, torch.ones_like(u), create_graph=True)[0]
    loss2 = (torch.squeeze(dudt) - 0)**2

    # compute physics loss
    u = pinn(t_physics)*torch.sin(a*t_physics+b)
    dudt = torch.autograd.grad(u, t_physics, torch.ones_like(u), create_graph=True)[0]
    d2udt2 = torch.autograd.grad(dudt, t_physics, torch.ones_like(dudt), create_graph=True)[0]
    loss3 = torch.mean((d2udt2 + mu*dudt + k*u)**2)

    # backpropagate joint loss, take optimiser step

    loss = loss1 + lambda1*loss2 + lambda2*loss3
    loss.backward()
    optimiser.step()

    # plot the result as training progresses
    if i % 500 == 0:
        if fig is None:
            fig = plt.figure(figsize=(8,3))
        fig.clf()
        u = (pinn(t_test)*torch.sin(a*t_test+b)).detach()
        plt.scatter(t_physics.detach()[:,0],
                    torch.zeros_like(t_physics)[:,0], s=20, lw=0, color="tab:green", alpha=0.6)
        plt.scatter(t_boundary.detach()[:,0],
                    torch.zeros_like(t_boundary)[:,0], s=20, lw=0, color="tab:red", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i} loss = %0.3f"%loss1.item())
        plt.legend()
        plt.show()
        display.display(fig)
        display.clear_output(wait=True)